# Libraries

In [1]:
import os
import shutil
import json
from sklearn.model_selection import train_test_split
from collections import Counter
from PIL import Image

# Helper Functions

In [2]:
def load_metadata(source_dir, metadata_filename):
    metadata_filePath = os.path.join(source_dir, metadata_filename)

    with open(metadata_filePath, 'r') as f:
        return json.load(f)

In [3]:
def save_metadata(output_dir, metadata_filename, metadata):
    metadata_outFilePath = os.path.join(output_dir, metadata_filename)

    with open(metadata_outFilePath, 'w') as f:
        json.dump(metadata, f, indent=4)

In [4]:
def performSplit(metadata, split_ratio):
    clusters = [entry['cluster'] for entry in metadata]
    
    activeLearning_data, prediction_data = train_test_split(
        metadata,
        test_size=split_ratio,
        random_state=42,
        stratify=clusters  # stratify 
        )
    
    return activeLearning_data, prediction_data

In [5]:
def copyframes(split):
    # destination path for the split frame
    out_path = os.path.join(split[2][1], split[0][0])
    os.makedirs(out_path, exist_ok=True)
    # save the metadata at the output
    save_metadata(split[2][1], split[0][1], split[1][0])

    if split[1][1]:
        for data in split[1][0]:
            frame_name = os.path.basename(data['image_path'])
            # src frame path
            src_framePath = os.path.join(split[2][0], frame_name)
            out_framePath = os.path.join(out_path, frame_name)
            # copy
            shutil.copy2(src_framePath, out_framePath)

    print(f"Copy complete for the split {split[0][1]} with images - {split[1][1]}")

In [6]:
def getCluster_ratio(data):
    # Count the number of frames per cluster
    cluster_counts = Counter(entry['cluster'] for entry in data)
    # Calculate total number of frames
    total_frames = len(data)
    # Print cluster percentages
    for cluster_id, count in cluster_counts.items():
        percentage = (count / total_frames) * 100
        print(f"Cluster {cluster_id}: {count} frames ({percentage:.2f}%)")

In [7]:
def printDetails(metadata, split_data):
    print("#############")
    print(f"Total frames: {len(metadata)}")
    getCluster_ratio(metadata)
    print("\t#############")
    for split in split_data:
        percentage = (len(split[1][0])/len(metadata))*100
        print(f"{split[0][1]} split: {len(split[1][0])} ({percentage:.2f}%)")
        getCluster_ratio(split[1][0])
        print("\t#############")


In [8]:
def copyframes_train_val(currentFrame_path, destination_dir):
    os.makedirs(destination_dir, exist_ok=True)
    shutil.copy2(currentFrame_path, destination_dir)


In [9]:
def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

In [10]:
def create_yolo_bBox_labels(annotation_bBox_info, frame_w, frame_h, class_label):
    yolo_bBox_label = []
    for bBox_info in annotation_bBox_info:
        bBox = bBox_info['bbox']

        x1, y1 = bBox["x1"], bBox["y1"]
        x2, y2 = bBox["x2"], bBox["y2"]

        # Convert to YOLO
        bBox_w = x2 - x1
        bBox_h = y2 - y1
        x_center = x1 + bBox_w / 2.0
        y_center = y1 + bBox_h / 2.0
        
        # Normalize
        x_center_norm = x_center / frame_w
        y_center_norm = y_center / frame_h
        w_norm = bBox_w / frame_w
        h_norm = bBox_h / frame_h
        
        # class_id, x_c, y_c, w, h
        yolo_line = f"{class_label} {x_center_norm:.6f} {y_center_norm:.6f} {w_norm:.6f} {h_norm:.6f}"
        
        for keypoint in bBox_info['keypoints'].values():
            kp_x = keypoint[0]/frame_w
            kp_y = keypoint[1]/frame_h
            kp_v = keypoint[2]

            keypoint_line = f" {kp_x:.6f} {kp_y:.6f} {kp_v}"

            yolo_line += keypoint_line
        
        yolo_bBox_label.append(yolo_line)
        
    return yolo_bBox_label


In [11]:
def save_yolo_label(trainLabels_dir, frame_name, yolo_bBox_labels):
    os.makedirs(trainLabels_dir, exist_ok=True)
    trainLabel_filename = os.path.join(trainLabels_dir, frame_name.replace("jpg", "txt"))

    with open(trainLabel_filename, 'w') as txt_out:
        txt_out.write("\n".join(yolo_bBox_labels))

In [12]:
def prepare_train_val(t_v_dir, annotations_dir, frame_name, annotation_info):
    t_v_images_dir = os.path.join(t_v_dir, "images")
    currentFrame_path = os.path.join(annotations_dir, frame_name)
    copyframes_train_val(currentFrame_path, t_v_images_dir)

    frame_w, frame_h = get_image_size(currentFrame_path)
    mouse_class_label = 0
    t_v_labels_dir = os.path.join(t_v_dir, "labels")
    yolo_bBox_labels = create_yolo_bBox_labels(annotation_info,  frame_w, frame_h, mouse_class_label)
    save_yolo_label(t_v_labels_dir, frame_name, yolo_bBox_labels)

In [13]:
def yolo_txt_to_annotation_json(
    txt_path, 
    image_filename,   # "image_filename.jpg"
    image_width, 
    image_height,
    keypoint_names=None
):
    """
    Reads a YOLO-like .txt (with bbox + 4 keypoints in normalized coords),
    and returns a dictionary in the original annotation style:

    {
      "image_filename": [
        {
          "bbox": {"x1":..., "y1":..., "x2":..., "y2":...},
          "keypoints": {
            "nose":  [...],
            "earL":  [...],
            "earR":  [...],
            "tailB": [...]
          }
        },
        ...
      ]
    }
    """
    if keypoint_names is None:
        # You can change the order or number of keypoints as needed:
        keypoint_names = ["nose", "earL", "earR", "tailB"]

    annotations = {image_filename: []}

    with open(txt_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        if not line:
            continue

        tokens = line.split()
        # The first 5 tokens are class_id, x_center, y_center, w, h
        class_id    = int(tokens[0])
        x_center_n  = float(tokens[1])
        y_center_n  = float(tokens[2])
        w_n         = float(tokens[3])
        h_n         = float(tokens[4])

        # Denormalize bounding box
        x_center = x_center_n * image_width
        y_center = y_center_n * image_height
        w        = w_n * image_width
        h        = h_n * image_height

        x1 = x_center - w / 2
        y1 = y_center - h / 2
        x2 = x_center + w / 2
        y2 = y_center + h / 2

        # Next tokens: each keypoint has x_kpt_n, y_kpt_n, v_kpt
        # For 4 keypoints, that's 12 tokens, starting at index = 5
        keypoints_dict = {}
        num_kpts = len(keypoint_names)
        
        # i.e. for 4 keypoints, range(4) => 0..3
        for i in range(num_kpts):
            x_kpt_n = float(tokens[5 + 3*i])
            y_kpt_n = float(tokens[5 + 3*i + 1])
            v_kpt   = float(tokens[5 + 3*i + 2])

            # denormalize
            x_kpt = x_kpt_n * image_width
            y_kpt = y_kpt_n * image_height

            kpt_name = keypoint_names[i]
            keypoints_dict[kpt_name] = [x_kpt, y_kpt, 2 if v_kpt > 0.5 else 1]

        annotations[image_filename].append({
            "bbox": {
                "x1": x1,
                "y1": y1,
                "x2": x2,
                "y2": y2
            },
            "keypoints": keypoints_dict,
            "mAnnotated": False
        })

    return annotations

# Main()

### Active learning and test split

In [14]:
# dirs' path
source_dir = "/mnt/c/Users/karti/chest/CNR/projects/data/neurocig/frames"
output_dir = "/mnt/c/Users/karti/chest/CNR/projects/data/neurocig/stratifySplit_frames"
metadata_filename_main = "frames_info.json"

In [ ]:
# load the frames metadata
metadata_main = load_metadata(source_dir, metadata_filename_main)

# Initial Split Percentages [active learning - 85, test - 15]
split_ratio_main = 0.15

# perform intial split
activeLearning_data, test_data = performSplit(metadata_main, split_ratio_main)
split_data_main = [(
        ('activeLearning', 'activeLearning.json'),
        (activeLearning_data, False),
        (source_dir, output_dir)
    ),
    (
        ('test', 'test.json'),
        (test_data, True),
        (source_dir, output_dir)
    )]

printDetails(metadata_main, split_data_main)

# copy the frames to their appropriate dirs
for split in split_data_main:
    copyframes(split)

### Inital split

In [15]:
activeLearning_dir = "/mnt/c/Users/karti/chest/CNR/projects/data/neurocig/stratifySplit_frames/activeLearning"
metadata_filename_al = "activeLearning.json"

In [ ]:
# number of initial frames of manuel annotation
intialN_annotatonData = 100

# load the active learning frames metadata
activeLearning_metadata = load_metadata(activeLearning_dir, metadata_filename_al)

# calculate the split ratio respective to intialN_annotatonData
split_ratio_aL = round(((intialN_annotatonData/len(activeLearning_metadata))*100)/100, 3)

# annotation split
activeLearning_metadata_new, annotation_metadata = performSplit(activeLearning_metadata, split_ratio_aL)

split_data_aL = (
        ('activeLearning', 'activeLearning.json'),
        (activeLearning_metadata_new, False),
        (source_dir, output_dir)
    )
    
split_data_annotations =   (
        ('annotations', 'annotations_metadataAL.json'),
        (annotation_metadata, True),
        (source_dir, activeLearning_dir)
    )

printDetails(activeLearning_metadata, [split_data_aL, split_data_annotations])


In [ ]:
prediction_percentage = 30
nOf_annotations = float((30*len(annotation_metadata))/100)
split_ratio_predict = round(((nOf_annotations/len(activeLearning_metadata_new))*100)/100, 3)

# annotation split
activeLearning_metadata_updated, predict_metadata = performSplit(activeLearning_metadata_new, split_ratio_predict)

split_data_aL = (
        ('activeLearning', 'activeLearning.json'),
        (activeLearning_metadata_updated, False),
        (source_dir, output_dir)
    )
    
split_data_predict =   (
        ('predict', 'predict.json'),
        (predict_metadata, True),
        (source_dir, activeLearning_dir)
    )

printDetails(activeLearning_metadata_new, [split_data_aL, split_data_predict])

In [ ]:
split_ratio_annotation = 0.15

# train val split
train_metadata, val_metadata = performSplit(annotation_metadata, split_ratio_annotation)

split_data_train = (
        ('train', 'train.json'),
        (train_metadata, False),
        (source_dir, activeLearning_dir)
    )
    
split_data_val =   (
        ('val', 'val.json'),
        (val_metadata, False),
        (source_dir, activeLearning_dir)
    )

printDetails(annotation_metadata, [split_data_train, split_data_val])

In [ ]:
split_data_is = [split_data_aL, split_data_annotations, split_data_train, split_data_val, split_data_predict]

for split in split_data_is:
    copyframes(split)

### Data Conversion
Annotations to train and val

In [16]:
train_dir = os.path.join(activeLearning_dir, "train")
train_json = "train.json"
val_dir = os.path.join(activeLearning_dir, "val")
val_json = "val.json"

# path to manual annotated frames and its json
annotations_dir = os.path.join(activeLearning_dir, "annotations")
annotation_json = "annotation.json"

In [ ]:
# manually annotated json
mAnnotated_json = load_metadata(annotations_dir, annotation_json)

# train and val frame metadata
train_metadata = load_metadata(train_dir, train_json)
val_metadata = load_metadata(val_dir, val_json)

In [ ]:
for frame_name, annotation_info in mAnnotated_json.items():
    for train_frame in train_metadata:
        if frame_name in train_frame['image_path']:
            prepare_train_val(train_dir, annotations_dir, frame_name, annotation_info)
            
    
    for val_frame in val_metadata:
        if frame_name in val_frame['image_path']:
            prepare_train_val(val_dir, annotations_dir, frame_name, annotation_info)

### Prediction Conversion

In [29]:
predict_metadata = load_metadata(activeLearning_dir, "predict.json")
predict_metadata
annotation_metadata = load_metadata(activeLearning_dir, "annotations_metadataAL.json")
annotation_metadata.extend(predict_metadata)
split_data_combine = (
        ('annotations', 'annotations_metadata.json'),
        (annotation_metadata, True),
        (source_dir, activeLearning_dir)
    )

copyframes(split_data_combine)

Copy complete for the split annotations_metadata.json with images - True


In [42]:
predict_dir = "/mnt/c/Users/karti/chest/CNR/projects/data/neurocig/stratifySplit_frames/activeLearning/predict"

combinedAnnotated_json = load_metadata(annotations_dir, annotation_json)
for label in os.listdir(predict_dir):
    if label.endswith(".txt"):
        label_path = os.path.join(predict_dir, label)

        image_name = label.replace('txt', 'jpg')
        img_path = os.path.join(predict_dir, image_name)
        img_w, img_h = get_image_size(img_path)
        
        predictions = yolo_txt_to_annotation_json(label_path, image_name,img_w, img_h, ["nose", "earL", "earR", "tailB"])
        combinedAnnotated_json.update(predictions)

save_metadata(annotations_dir, "annotation.json", combinedAnnotated_json)

In [23]:
predict_metadata = load_metadata(activeLearning_dir, "predict.json")

combinedAnnotated_json = load_metadata(annotations_dir, annotation_json)
predict_metadata

[{'video': 'Gabbia1-D21-Cig(1)-pre.mp4',
  'frame_index': 8900,
  'cluster': 0,
  'image_path': '/home/jalal/projects/data/neurocig/frames/Gabbia1-D21-Cig(1)-pre_frame_8900.jpg'},
 {'video': 'Gabbia2-D16-Cig(1)-pre.mp4',
  'frame_index': 7850,
  'cluster': 0,
  'image_path': '/home/jalal/projects/data/neurocig/frames/Gabbia2-D16-Cig(1)-pre_frame_7850.jpg'},
 {'video': 'Gabbia1-D31-eCig(1)-pre.mp4',
  'frame_index': 2250,
  'cluster': 3,
  'image_path': '/home/jalal/projects/data/neurocig/frames/Gabbia1-D31-eCig(1)-pre_frame_2250.jpg'},
 {'video': 'Gabbia1-D21-aria(1).mp4',
  'frame_index': 5950,
  'cluster': 0,
  'image_path': '/home/jalal/projects/data/neurocig/frames/Gabbia1-D21-aria(1)_frame_5950.jpg'},
 {'video': 'Gabbia2-D11-aria(3).mp4',
  'frame_index': 2750,
  'cluster': 2,
  'image_path': '/home/jalal/projects/data/neurocig/frames/Gabbia2-D11-aria(3)_frame_2750.jpg'},
 {'video': 'Gabbia2-D4-Cig(3)-pre.mp4',
  'frame_index': 6650,
  'cluster': 4,
  'image_path': '/home/jalal/pr

In [27]:
for filename, annotations_data in combinedAnnotated_json.items():
    mAnnotated_flag = True
    for predict_frames in predict_metadata:
        if filename in predict_frames['image_path']:
            mAnnotated_flag = False
            break
        
    for mice_annotation in annotations_data:
        mice_annotation["mAnnotated"] = mAnnotated_flag

In [28]:
combinedAnnotated_json

{'Gabbia1-D1-aria(1)_frame_6000.jpg': [{'bbox': {'x1': 397.0103759765625,
    'y1': 237.75,
    'x2': 539.0103759765625,
    'y2': 323.75},
   'keypoints': {'nose': [511.0103759765625, 259.75, 2],
    'earL': [497.0103759765625, 262.75, 2],
    'earR': [504.0103759765625, 281.75, 2],
    'tailB': [419.0103759765625, 293.75, 2]},
   'mAnnotated': True},
  {'bbox': {'x1': 189.0103759765625,
    'y1': 234.75,
    'x2': 314.0103759765625,
    'y2': 315.75},
   'keypoints': {'nose': [201.0103759765625, 271.75, 2],
    'earL': [217.0103759765625, 284.75, 2],
    'earR': [217.0103759765625, 266.75, 2],
    'tailB': [297.0103759765625, 269.75, 2]},
   'mAnnotated': True},
  {'bbox': {'x1': 64.0103759765625,
    'y1': 241.75,
    'x2': 163.0103759765625,
    'y2': 354.75},
   'keypoints': {'nose': [103.0103759765625, 341.75, 2],
    'earL': [117.0103759765625, 312.75, 2],
    'earR': [89.0103759765625, 306.75, 2],
    'tailB': [128.0103759765625, 250.75, 2]},
   'mAnnotated': True},
  {'bbox': 

In [29]:
save_metadata(annotations_dir, "annotation.json", combinedAnnotated_json)

### Active learning split